In [ ]:
list_td = ['nmod:npmod', 'det:predet', 'obl:npmod', 'acl', 'acl:relcl', 'advcl', 'advmod', 'advmod:emph', 'advmod:lmod', 'amod', 'appos', 'aux', 'aux:pass', 'case', 'cc', 'cc:preconj', 'ccomp', 'clf', 'compound', 'compound:lvc', 'compound:prt', 'compound:redup', 'compound:svc', 'conj', 'cop', 'csubj', 'csubj:pass', 'dep', 'det', 'det:numgov', 'det:nummod', 'det:poss', 'discourse', 'dislocated', 'expl', 'expl:impers', 'expl:pass', 'expl:pv', 'fixed', 'flat', 'flat:foreign', 'flat:name', 'goeswith', 'iobj', 'list', 'mark', 'nmod', 'nmod:poss', 'nmod:tmod', 'nsubj', 'nsubj:pass', 'nummod', 'nummod:gov', 'obj', 'obl', 'obl:agent', 'obl:arg', 'obl:lmod', 'obl:tmod', 'orphan', 'parataxis', 'punct', 'reparandum', 'vocative', 'xcomp']

In [ ]:
!pip install contractions
!pip install emoji
!pip install lexical-diversity
!pip install stanza

In [ ]:
import re
import pandas as pd
import contractions
from emoji import UNICODE_EMOJI
import emoji
from html import unescape
from statistics import pstdev
import numpy as np
from lexical_diversity import lex_div as ld
from sklearn.preprocessing import normalize
import nltk
from nltk.corpus import stopwords, wordnet, sentiwordnet
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk import sent_tokenize, word_tokenize, pos_tag
import pickle
import stanza
from scipy import sparse
from sklearn.preprocessing import normalize
import os
from sklearn import preprocessing
from sklearn.model_selection import ParameterGrid
import time
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from pathlib import Path
import xml.etree.ElementTree as ET
import joblib
from sklearn.linear_model import RidgeClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import make_scorer
import zipfile

stanza.download('en')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('sentiwordnet')

In [ ]:
r = open('/content/drive/My Drive/datasets/pan21-author-profiling-training-2021-03-14/en/truth.txt', "r")
data = r.read().split("\n")
id = []
hate_speech = []

for line in data:
  l = line.split(":::")
  if len(l)>1:
    id.append(l[0])
    hate_speech.append(l[1])

labels_data = pd.DataFrame()
labels_data["ID"] = id
labels_data["Hate_speech"] = hate_speech

# reading in and concatenating English tweets
pathlist = Path('/content/drive/My Drive/datasets/pan21-author-profiling-training-2021-03-14/en').glob('**/*.xml')
id = []
tweets_raw_lists, tweets_raw_concat = [], []
for path in pathlist:  #iterate files
  head, tail = os.path.split(path)
  author = tail.split(".")[0]
  id.append(author)
  path_in_str = str(path)
  tree = ET.parse(path_in_str)
  docs = tree.getroot().getchildren()[0]
  tweets = []
  for tweet in docs:
    tweets.append(tweet.text)
  tweets_raw_lists.append(tweets)
  tweets_raw_concat.append(' '.join(tweets).lower())

text_data_lists = pd.DataFrame()
text_data_lists["ID"] = id
text_data_lists["Tweets"] = tweets_raw_lists

text_data_concat = pd.DataFrame()
text_data_concat["ID"] = id
text_data_concat["Tweets"] = tweets_raw_concat

# merging data to one dataframe
tweets_data_lists = pd.merge(labels_data, text_data_lists, how = 'inner', on = 'ID')
tweets_data_lists.to_csv('/content/drive/My Drive/NLPproject/data/lists_tweets_data.tsv', sep = '\t', index = False)

tweets_data_concat = pd.merge(labels_data, text_data_concat, how = 'inner', on = 'ID')
tweets_data_concat.to_csv('/content/drive/My Drive/NLPproject/data/concatenated_tweets_data.tsv', sep = '\t', index = False)

In [ ]:
def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return None

negations = set(['not', 'never', 'none', 'nobody', 'nowhere', 'nothing', 'neither', 'no', 'noone'])

def negation(tokens):
  negations_booleans = []
  boolean = False
  for token in tokens:
    token_boolean = False
    if token in negations:
      if boolean:
        boolean = False
      else:
        boolean = True
    elif re.match(r'[\.:;!?]+$', token):
      boolean = False
    elif boolean:
      token_boolean = True
    negations_booleans.append(token_boolean)
  return negations_booleans

def rescale(value, min, max):
  scale = 1/(max - min)
  return (value - min) * scale

def negate_(value, negate):
  result = value
  if negate:
    if value > 0.5:
      result = value - ((value - 0.5) * 2)
    else:
      result = ((0.5 - value) * 2) + value
  return np.round(result, 3)

def sentiment_value(word, negate, wntag):
  synsets = wordnet.synsets(word, pos = wntag)
  if len(synsets) != 0:
    synset = synsets[0]
    swn_synset = sentiwordnet.senti_synset(synset.name())
    pos, neg = swn_synset.pos_score(), swn_synset.neg_score()
    score = pos - neg
    if score == 0:
      return True, -1
    else:
      result = np.round(rescale(score, -1., 1.), 3)
      return True, negate_(result, negate)

  sentiment = sentiment_dict.get(word)
  if sentiment is not None:
    if sentiment == 0.5:
      return True, -1
    return True, negate_(sentiment, negate)

  return False, "unknown"

def lemma_(word, wntag):
  if wntag is None:
    lemma = lem.lemmatize(word) 
  else:
    lemma = lem.lemmatize(word, pos = wntag)
  return lemma

def dict_emoticons():
  return {":\)+":"happy", ":\(+":"sad", ":[Dd]+":"happy", "=\)+":"happy", ":[pP]+":"playful", ":3+":"happy", 
          "[Xx][dD]+":"happy_laugh", ">:\(+":"angry", ":'\(+":"sad_cry", "[Xx][Pp]+":"playful"}

def emoticon(word):
  dict_emoticon_patterns = dict_emoticons()
  dict_list = list(dict_emoticon_patterns.keys())
  for emoticon in dict_list:
    if re.search(f"^{emoticon}", word):
      return re.sub(f"^{emoticon}", dict_emoticon_patterns[emoticon], word)
  return word

def demojify(word):
  new_string = []
  boolean = True
  for char in word:
    if char in UNICODE_EMOJI["en"]:
      new_string.append(' ' + char + ' ')
      boolean = False
    else:
      new_string.append(char)
  if boolean:
    return word
  new_string = re.sub(r' +', ' ', ''.join(new_string)).strip()
  return emoji.demojize(new_string)

def parse(sentence, nlp, td_matrix, sample):
  sentence = re.sub(r'[^0-9A-Za-z _!.?,();"\'\\%$&~/-]', '', sentence)
  sentence = re.sub(r' +', ' ', sentence).strip()
  doc = nlp(sentence)
  for sent in doc.sentences:
    typed_dependancies = [tup[1] for tup in sent.dependencies]
    #print(typed_dependancies)
    for td in sent.dependencies:
      if td[1] == "root":
        continue
      if td[1] not in set_td:
        continue
      index = dict_td[td[1]]
      value = td_matrix[sample, index]
      td_matrix[sample, index] = value + 1
  return td_matrix


def prep_and_extraction(data, td_bool):
  
  all_tweets = data["Tweets"].tolist()
  cleaned_tweets = []
  dict_ = {}

  boolean = True
  sd_len_char_acc, sd_len_word_acc, min_len_char_acc, min_len_word_acc, max_len_char_acc, max_len_word_acc = [], [], [], [], [], []
  rng_len_char_acc, rng_len_word_acc, mean_len_char_acc, mean_len_word_acc = [], [], [], []
  rt_acc, url_acc, hash_acc, user_acc = [], [], [], []
  ttr_acc = []
  pos_acc, neg_acc, neu_acc, unk_acc, sent_mean_acc, sent_std_acc, hate_acc = [], [], [], [], [], [], []
  counter = 0
  if td_bool:
    nlp = stanza.Pipeline('en')
    typedep_matrix = np.zeros((data.shape[0], len(dict_td)), dtype = np.double)

  for account_feed in all_tweets:
    print(counter)
    cleaned_feed = []

    char_lenghts, word_lenghts, ttr_ = [], [], []
    rts, urls, hashs, users = 0, 0, 0, 0
    positives, negatives, neutrals, unknowns, hatefulls = 0, 0, 0, 0, 0
    overall_sent = []

    for tweet in account_feed:

      tweet = unescape(tweet)

      ttr_.extend(ld.flemmatize(tweet))
      
      char_lenghts.append(len(tweet))

      words = []
      tweet_list = tweet.split()

      #if boolean:
        #print(tweet)
        #print(len(tweet))
        #print(len(tweet_list))
      word_lenghts.append(len(tweet_list))

      for word in tweet_list:
      
        #word = word.lower()
        #new_word = contractions.fix(word).lower()
        new_word = contractions.fix(word)
        if new_word != word:
          dict_[word] = new_word

        if new_word != "":
          words.append(new_word)
      tweet = ' '.join(words)
      
      words = []
      for_parsing = []
      tweet_list = tweet.split()

      for word in tweet_list:

        parsing_word = word
        new_word = demojify(word)

        if not new_word.isalnum():
          new_word = emoticon(new_word)
          
          if re.search("#USER#", new_word):
            new_word = re.sub("#USER#", "", new_word)
            parsing_word = re.sub("#USER#", "", parsing_word)
            users += 1
          
          if re.search("#URL#", new_word):
            new_word = re.sub("#URL#", "", new_word)
            parsing_word = re.sub("#URL#", "", parsing_word)
            urls += 1

          if re.search("#HASHTAG#", new_word):
            new_word = re.sub("#HASHTAG#", "", new_word)
            parsing_word = re.sub("#HASHTAG#", "", parsing_word)
            hashs += 1
        
        if re.match("RT$", new_word):
          new_word = re.sub("RT", "", new_word)
          parsing_word = re.sub("RT", "", parsing_word)
          rts += 1

        if new_word != "":
          words.append(new_word)
        for_parsing.append(parsing_word)
      tweet = ' '.join(words)

      if td_bool:
        typedep_matrix = parse(' '.join(for_parsing), nlp, typedep_matrix, counter)

      tweet = tweet.lower()
      
      sentiment = 0.
      sentences = sent_tokenize(tweet)
      for sent in sentences:

        sent = re.sub("_|-", " ", sent)
        words = word_tokenize(sent)
        tagged = nltk.pos_tag(words)

        stemmed_words = [ps.stem(word) for word in words]
        stemmed_tagged = nltk.pos_tag(stemmed_words)

        booleans = negation(words)

        for i in range(len(words)):
          
          word = words[i]
          word = re.sub('[^0-9a-z]', "", word)
          
          if word != '' and word not in set_stopwords:
            wntag = get_wordnet_pos(tagged[i][1])
            
            exists, score = sentiment_value(word, booleans[i], wntag)
            hatefull = word in hatefull_set

            if not exists:
              lemma = lemma_(word, wntag)
              exists, score = sentiment_value(lemma, booleans[i], wntag)
            if not hatefull:
              lemma = lemma_(word, wntag)
              hatefull = lemma in hatefull_set
            
            if not exists:
              exists, score = sentiment_value(stemmed_words[i], booleans[i], get_wordnet_pos(stemmed_tagged[i][1]))
            if not hatefull:
              hatefull = stemmed_words[i] in hatefull_set
              
            if not exists:
              if not hatefull:
                unknowns += 1
            else:
              if score == -1:
                neutrals += 1
              elif score > 0.5:
                positives += 1
                sentiment += score
              else:
                negatives += 1
                sentiment += score
              
            if hatefull:
              hatefulls += 1
      overall_sent.append(sentiment)

      tweet = re.sub('[^a-z _-]', "", tweet)

      words = []
      tweet_list = tweet.split()
      for word in tweet_list:

        new_word = re.sub('[0-9]+', 'num', word)

        if new_word != "":
          words.append(new_word)
      tweet = ' '.join(words)

      tweet = re.sub(r' +', ' ', tweet)
      if boolean:
        #print(tweet)
        boolean = False

      cleaned_feed.append(tweet)
    cleaned_tweets.append(cleaned_feed)

    sd_len_char_acc.append(pstdev(char_lenghts))
    sd_len_word_acc.append(pstdev(word_lenghts))
    min_char = min(char_lenghts)
    min_len_char_acc.append(min_char)
    min_word = min(word_lenghts)
    min_len_word_acc.append(min_word)
    max_char = max(char_lenghts)
    max_len_char_acc.append(max_char)
    max_word = max(word_lenghts)
    max_len_word_acc.append(max_word)
    rng_len_char_acc.append(max_char - min_char)
    rng_len_word_acc.append(max_word - min_word)
    mean_len_char_acc.append(np.mean(char_lenghts))
    mean_len_word_acc.append(np.mean(word_lenghts))

    rt_acc.append(rts)
    url_acc.append(urls)
    hash_acc.append(hashs)
    user_acc.append(users)

    ttr_acc.append(ld.ttr(ttr_))

    pos_acc.append(positives)
    neg_acc.append(negatives)
    neu_acc.append(neutrals)
    unk_acc.append(unknowns)
    sent_std_acc.append(pstdev(overall_sent))
    sent_mean_acc.append(np.mean(overall_sent))
    hate_acc.append(hatefulls)

    counter += 1

  #print(dict_.items())
  #print(cleaned_tweets[0])
  #print(sd_len_char_acc)

  data["Tweets"] = cleaned_tweets

  stacked = np.column_stack((sd_len_char_acc, sd_len_word_acc, min_len_char_acc, min_len_word_acc, max_len_char_acc, max_len_word_acc, rng_len_char_acc, rng_len_word_acc, mean_len_char_acc, mean_len_word_acc, rt_acc, url_acc, hash_acc, user_acc, ttr_acc, pos_acc, neg_acc, neu_acc, unk_acc, sent_std_acc, sent_mean_acc, hate_acc))
  stan_scaler = preprocessing.StandardScaler()
  scaled = stan_scaler.fit_transform(stacked)
  #print(stacked)
  #print(scaled)
  #print(stan_scaler.mean_)
  #print(stan_scaler.scale_)
  pickle.dump(stan_scaler, open('/content/drive/My Drive/NLPproject/models/stan_scaler.pickle', 'wb'))

  data["sd_len_char_acc"] = scaled[:, 0].tolist()
  data["sd_len_word_acc"] = scaled[:, 1].tolist()
  data["min_len_char_acc"] = scaled[:, 2].tolist()
  data["min_len_word_acc"] = scaled[:, 3].tolist()
  data["max_len_char_acc"] = scaled[:, 4].tolist()
  data["max_len_word_acc"] = scaled[:, 5].tolist()
  data["rng_len_char_acc"] = scaled[:, 6].tolist()
  data["rng_len_word_acc"] = scaled[:, 7].tolist()
  data["mean_len_char_acc"] = scaled[:, 8].tolist()
  data["mean_len_word_acc"] = scaled[:, 9].tolist()

  data["rt_acc"] = scaled[:, 10].tolist()
  data["url_acc"] = scaled[:, 11].tolist()
  data["hash_acc"] = scaled[:, 12].tolist()
  data["user_acc"] = scaled[:, 13].tolist()

  data["ttr_acc"] = scaled[:, 14].tolist()

  data["pos_acc"] = scaled[:, 15].tolist()
  data["neg_acc"] = scaled[:, 16].tolist()
  data["neu_acc"] = scaled[:, 17].tolist()
  data["unk_acc"] = scaled[:, 18].tolist()
  data["sent_std_acc"] = scaled[:, 19].tolist()
  data["sent_mean_acc"] = scaled[:, 20].tolist()
  data["hate_acc"] = scaled[:, 21].tolist()

  if td_bool:
    typedep_matrix = normalize(typedep_matrix, norm = 'l2', axis = 1)
    typedep_matrix = sparse.csr_matrix(typedep_matrix)
    return data, typedep_matrix
  else:
    return data

lists_tweets_data = pd.read_csv('/content/drive/My Drive/NLPproject/data/lists_tweets_data.tsv', delimiter = '\t', encoding = 'utf-8', converters = {'Tweets': eval})

lem = WordNetLemmatizer()
ps = PorterStemmer()
set_stopwords = set(stopwords.words('english'))

a_file = open("/content/drive/My Drive/NLPproject/additional_stuff/sentiment_dict.pkl", "rb")
sentiment_dict = pickle.load(a_file)

hatefull_df = pd.read_csv('/content/drive/My Drive/NLPproject/additional_stuff/hatebase_vocab.csv')
hatefull_set = set()
for term in hatefull_df['term']:
  hatefull_set.add(term.lower())
print(hatefull_set)

td_matrix = 0
if os.path.exists('/content/drive/My Drive/NLPproject/models/td_matrix.pkl'):
  with open('/content/drive/My Drive/NLPproject/models/td_matrix.pkl', 'rb') as f:
    td_matrix = pickle.load(f)
    prepared_data = prep_and_extraction(lists_tweets_data, False)
else:
  list_td = ['nmod:npmod', 'det:predet', 'obl:npmod', 'acl', 'acl:relcl', 'advcl', 'advmod', 'advmod:emph', 'advmod:lmod', 'amod', 'appos', 'aux', 'aux:pass', 'case', 'cc', 'cc:preconj', 'ccomp', 'clf', 'compound', 'compound:lvc', 'compound:prt', 'compound:redup', 'compound:svc', 'conj', 'cop', 'csubj', 'csubj:pass', 'dep', 'det', 'det:numgov', 'det:nummod', 'det:poss', 'discourse', 'dislocated', 'expl', 'expl:impers', 'expl:pass', 'expl:pv', 'fixed', 'flat', 'flat:foreign', 'flat:name', 'goeswith', 'iobj', 'list', 'mark', 'nmod', 'nmod:poss', 'nmod:tmod', 'nsubj', 'nsubj:pass', 'nummod', 'nummod:gov', 'obj', 'obl', 'obl:agent', 'obl:arg', 'obl:lmod', 'obl:tmod', 'orphan', 'parataxis', 'punct', 'reparandum', 'vocative', 'xcomp']
  dict_td = {}
  for i in range(len(list_td)):
    dict_td[list_td[i]] = i
  set_td = set(list_td)
  prepared_data, td_matrix = prep_and_extraction(lists_tweets_data, True)
  with open('/content/drive/My Drive/NLPproject/models/td_matrix.pkl', 'wb') as f:
    pickle.dump(td_matrix, f)

print(td_matrix)
prepared_data.to_csv('/content/drive/My Drive/NLPproject/data/prepared_data_v7.tsv', sep = '\t', index = False)


---

---

---

---

---
---


---
---

---

---

---

---
---


---

---



---
---



In [ ]:
def build_df(data, n_gram):
  df = {}
  for account_feed in data:
    account_set = set()
    for tweet in account_feed:
      words = re.split(r"[\s]+", tweet.strip())

      if n_gram == 0:
        both_df(words, account_set, df)
      elif n_gram == 1:
        unigrams_df(words, account_set, df)
      else:
        bigrams_df(words, account_set, df)
  return df

def unigrams_df(words, account_set, df):
  for word in words:
    if word not in account_set and word not in set_stopwords:
      df[word] = df.get(word, 0) + 1
      account_set.add(word)

def bigrams_df(words, account_set, df):
  previous = ''
  for word in words:
    if previous != '':
      bigram = previous + ' ' + word
      if bigram not in account_set:
        df[bigram] = df.get(bigram, 0) + 1
        account_set.add(bigram)
    previous = word

def both_df(words, account_set, df):
  previous = ''
  for word in words:
    if previous != '':
      bigram = previous + ' ' + word
      if bigram not in account_set:
        df[bigram] = df.get(bigram, 0) + 1
        account_set.add(bigram)
    if word not in account_set and word not in set_stopwords:
      df[word] = df.get(word, 0) + 1
      account_set.add(word)
    previous = word

def unigrams_tf(words, tf):
  for word in words:
    if word not in set_stopwords:
      tf[word] = tf.get(word, 0) + 1

def bigrams_tf(words, tf):
  previous = ''
  for word in words:
    if previous != '':
      bigram = previous + ' ' + word
      tf[bigram] = tf.get(bigram, 0) + 1
    previous = word

def both_tf(words, tf):
  previous = ''
  for word in words:
    if previous != '':
      bigram = previous + ' ' + word
      tf[bigram] = tf.get(bigram, 0) + 1
    if word not in set_stopwords:
      tf[word] = tf.get(word, 0) + 1
    previous = word

def build_tfidf(data, df, vocab_set, vocab_dict, n_gram):
  tfidf_matrix = np.zeros((data.shape[0], len(df)), dtype = np.double)

  sample = 0
  for account_feed in data:
    account_tf = {}

    for tweet in account_feed:
      words = re.split(r"[\s]+", tweet.strip())
      if n_gram == 0:
        both_tf(words, account_tf)
      elif n_gram == 1:
        unigrams_tf(words, account_tf)
      else:
        bigrams_tf(words, account_tf)
    
    for key, value in account_tf.items():
      #print(tf.items())
      if key not in vocab_set:
        continue
      index = vocab_dict[key]
      tf_idf = (1. + np.log(value)) * (np.log((1. + data.shape[0]) / (1. + df[key])) + 1.)
      tfidf_matrix[sample, index] = tf_idf
    sample += 1

  tfidf_matrix = normalize(tfidf_matrix, norm = 'l2', axis = 1)
  return tfidf_matrix

def fit(data, n_gram, min_df):
  df = build_df(data, n_gram)
  df = {key:val for key, val in df.items() if val >= min_df}
  vocab_dict = {}
  vocab = list(df.keys())
  for i in range(len(vocab)):
    vocab_dict[vocab[i]] = i
  vocab_set = set(vocab)
  #print(vocab_set)
  return df, vocab_set, vocab_dict 

def transform(data, n_gram, df, vocab_set, vocab_dict):
  tfidf_matrix = build_tfidf(data, df, vocab_set, vocab_dict, n_gram)
  tfidf_matrix = sparse.csr_matrix(tfidf_matrix)
  #print(tfidf_matrix.getnnz() / np.prod(tfidf_matrix.shape), " %")
  return tfidf_matrix

#def combine(m1, m2, indexes):
def combine(m1, m2):
  word_char = sparse.hstack((m1, m2)).tocsr()
  #td_mat_slice = td_matrix[indexes, :]
  #result = sparse.hstack((word_char, td_mat_slice)).tocsr()
  result = sparse.hstack((word_char, td_matrix)).tocsr()
  return result

In [ ]:
# -----------------------------------------------------------------------------------------------------------------------------
# -----------------------------------------------------------------------------------------------------------------------------
# -----------------------------------------------------------------------------------------------------------------------------

data = pd.read_csv('/content/drive/My Drive/NLPproject/data/prepared_data_v7.tsv', delimiter = '\t', encoding = 'utf-8', converters = {'Tweets': eval})
data_tweets = data["Tweets"]
labels = data["Hate_speech"]

data_char = pd.read_csv('/content/drive/My Drive/NLPproject/data/concatenated_tweets_data.tsv', delimiter = '\t', encoding = 'utf-8')
data_tweets_char = data_char["Tweets"]

grid = ParameterGrid({"ngram__range":[1, 0, 2], "min__df": [3, 4, 5, 6, 7], "cngram__range":[(2, 2), (3, 3), (4, 4)], "cmin__df":[3, 4, 5, 6, 7]})
#svm_v1m_df = pd.DataFrame()
clr_df = pd.DataFrame()
params_list = []
scores_list = []
set_stopwords = set(stopwords.words('english'))
pd.set_option("display.max_rows", None, "display.max_columns", None)

counter = 1
print(time.strftime("%m/%d/%Y %H:%M:%S"))
for params in grid:
  pipeline = Pipeline([('svm', svm.SVC(kernel = 'linear', random_state = 5, verbose = False))])
  parameters = {"svm__C":[1, 10, 100, 1000]}
  grid_search = GridSearchCV(pipeline, parameters, cv = StratifiedKFold(5, shuffle = True, random_state = 0), n_jobs = -1)

  df, vocab_set, vocab_dict = fit(data_tweets, params["ngram__range"], params["min__df"])
  data_train_tr = transform(data_tweets, params["ngram__range"], df, vocab_set, vocab_dict)
  char_vectorizer = TfidfVectorizer(ngram_range = params["cngram__range"], min_df = params["cmin__df"], use_idf = True, smooth_idf = True, sublinear_tf = True, analyzer = 'char_wb')
  data_train_char_tr = char_vectorizer.fit_transform(data_tweets_char)
  #f_data_train_tr = combine(data_train_tr, data_train_char_tr, train_index)
  f_data_train_tr = combine(data_train_tr, data_train_char_tr)
  print(f_data_train_tr.shape, data_train_char_tr.shape, data_train_tr.shape, td_matrix.shape)

  print("Fitting...")
  #svm2 = grid_search.fit(final_data.iloc[:,:], merged.iloc[:,2:3])
  clr = grid_search.fit(f_data_train_tr, labels)
  #svm_v1 = grid_search.fit(tfidf_matrix, data_v1['Hate_speech'])
  #print(svm_v1.cv_results_['params'])
  for params_ in clr.cv_results_['params']:
    params_.update(params)
    params_list.append(params_)
  #print(params_list)
  #print(svm_v1.cv_results_['mean_test_score'])
  scores_list.extend(clr.cv_results_['mean_test_score'])
  #print(scores_list)
  print(counter)
  print(time.strftime("%m/%d/%Y %H:%M:%S"))
  counter += 1

clr_df['params'] = params_list
clr_df['scores'] = scores_list
clr_df.to_csv('/content/drive/My Drive/NLPproject/results/F_svm1.tsv', sep = '\t', index = False)

05/22/2021 19:54:54
(200, 10896) (200, 2515) (200, 8316) (200, 65)
Fitting...
1
05/22/2021 19:55:03
(200, 28764) (200, 2515) (200, 26184) (200, 65)
Fitting...
2
05/22/2021 19:55:16
(200, 20448) (200, 2515) (200, 17868) (200, 65)
Fitting...
3
05/22/2021 19:55:25
(200, 9148) (200, 2515) (200, 6568) (200, 65)
Fitting...
4
05/22/2021 19:55:33
(200, 20932) (200, 2515) (200, 18352) (200, 65)
Fitting...
5
05/22/2021 19:55:45
(200, 14364) (200, 2515) (200, 11784) (200, 65)
Fitting...
6
05/22/2021 19:55:54
(200, 8036) (200, 2515) (200, 5456) (200, 65)
Fitting...
7
05/22/2021 19:56:02
(200, 16686) (200, 2515) (200, 14106) (200, 65)
Fitting...
8
05/22/2021 19:56:13
(200, 11230) (200, 2515) (200, 8650) (200, 65)
Fitting...
9
05/22/2021 19:56:22
(200, 7276) (200, 2515) (200, 4696) (200, 65)
Fitting...
10
05/22/2021 19:56:30
(200, 13992) (200, 2515) (200, 11412) (200, 65)
Fitting...
11
05/22/2021 19:56:41
(200, 9296) (200, 2515) (200, 6716) (200, 65)
Fitting...
12
05/22/2021 19:56:49
(200, 6726) (20

In [ ]:
#SVM
# {'svm__C': 1, 'cmin__df': 5, 'cngram__range': (2, 2), 'min__df': 5, 'ngram__range': 1} 0.6599999999999999
df, vocab_set, vocab_dict = fit(data_tweets, 1, 5)
pickle.dump(df, open('/content/drive/My Drive/NLPproject/final/features/svm_df.pkl', 'wb'))
pickle.dump(vocab_set, open('/content/drive/My Drive/NLPproject/final/features/svm_vocab_set.pkl', 'wb'))
pickle.dump(vocab_dict, open('/content/drive/My Drive/NLPproject/final/features/svm_vocab_dict.pkl', 'wb'))

data_train_tr = transform(data_tweets, 1, df, vocab_set, vocab_dict)
char_vectorizer = TfidfVectorizer(ngram_range = (2, 2), min_df = 5, use_idf = True, smooth_idf = True, sublinear_tf = True, analyzer = 'char_wb')
data_train_char_tr = char_vectorizer.fit_transform(data_tweets_char)
pickle.dump(char_vectorizer, open('/content/drive/My Drive/NLPproject/final/features/svm_char_vectorizer.pkl', 'wb'))

f_data_train_tr = combine(data_train_tr, data_train_char_tr)
clr = svm.SVC(C = 1, kernel = 'linear', random_state = 5, verbose = False, probability = True)
clr.fit(f_data_train_tr, labels)
pickle.dump(clr, open('/content/drive/My Drive/NLPproject/final/models/svm_1.sav', 'wb'))

In [ ]:
# -----------------------------------------------------------------------------------------------------------------------------
# -----------------------------------------------------------------------------------------------------------------------------
# -----------------------------------------------------------------------------------------------------------------------------

data = pd.read_csv('/content/drive/My Drive/NLPproject/data/prepared_data_v7.tsv', delimiter = '\t', encoding = 'utf-8', converters = {'Tweets': eval})
data_tweets = data["Tweets"]
labels = data["Hate_speech"]

data_char = pd.read_csv('/content/drive/My Drive/NLPproject/data/concatenated_tweets_data.tsv', delimiter = '\t', encoding = 'utf-8')
data_tweets_char = data_char["Tweets"]

grid = ParameterGrid({"ngram__range":[1, 0, 2], "min__df": [4, 5, 6, 7], "cngram__range":[(2, 2), (3, 3), (4, 4)], "cmin__df":[6, 7, 8, 9]})
#svm_v1m_df = pd.DataFrame()
clr_df = pd.DataFrame()
params_list = []
scores_list = []
set_stopwords = set(stopwords.words('english'))
pd.set_option("display.max_rows", None, "display.max_columns", None)

counter = 1
print(time.strftime("%m/%d/%Y %H:%M:%S"))
for params in grid:
  #pipeline = Pipeline([('lr', LogisticRegression(penalty = 'l2', solver = 'liblinear', fit_intercept = False, verbose = 0, random_state = 5))])
  pipeline = Pipeline([('lr', LogisticRegression(penalty = 'l2', solver = 'liblinear', verbose = 0, random_state = 5))])
  parameters = {"lr__C":[.1, 1, 10, 100, 1000]}
  grid_search = GridSearchCV(pipeline, parameters, cv = StratifiedKFold(5, shuffle = True, random_state = 0), n_jobs = -1)

  df, vocab_set, vocab_dict = fit(data_tweets, params["ngram__range"], params["min__df"])
  data_train_tr = transform(data_tweets, params["ngram__range"], df, vocab_set, vocab_dict)
  char_vectorizer = TfidfVectorizer(ngram_range = params["cngram__range"], min_df = params["cmin__df"], use_idf = True, smooth_idf = True, sublinear_tf = True, analyzer = 'char_wb')
  data_train_char_tr = char_vectorizer.fit_transform(data_tweets_char)
  #f_data_train_tr = combine(data_train_tr, data_train_char_tr, train_index)
  f_data_train_tr = combine(data_train_tr, data_train_char_tr)
  print(f_data_train_tr.shape, data_train_char_tr.shape, data_train_tr.shape, td_matrix.shape)

  print("Fitting...")
  #svm2 = grid_search.fit(final_data.iloc[:,:], merged.iloc[:,2:3])
  clr = grid_search.fit(f_data_train_tr, labels)
  #svm_v1 = grid_search.fit(tfidf_matrix, data_v1['Hate_speech'])
  #print(svm_v1.cv_results_['params'])
  for params_ in clr.cv_results_['params']:
    params_.update(params)
    params_list.append(params_)
  #print(params_list)
  #print(svm_v1.cv_results_['mean_test_score'])
  scores_list.extend(clr.cv_results_['mean_test_score'])
  #print(scores_list)
  print(counter)
  print(time.strftime("%m/%d/%Y %H:%M:%S"))
  counter += 1

clr_df['params'] = params_list
clr_df['scores'] = scores_list
clr_df.to_csv('/content/drive/My Drive/NLPproject/results/F_lr1.tsv', sep = '\t', index = False)

05/22/2021 23:00:16
(200, 8446) (200, 1813) (200, 6568) (200, 65)
Fitting...
1
05/22/2021 23:00:20
(200, 20230) (200, 1813) (200, 18352) (200, 65)
Fitting...
2
05/22/2021 23:00:25
(200, 13662) (200, 1813) (200, 11784) (200, 65)
Fitting...
3
05/22/2021 23:00:29
(200, 7334) (200, 1813) (200, 5456) (200, 65)
Fitting...
4
05/22/2021 23:00:33
(200, 15984) (200, 1813) (200, 14106) (200, 65)
Fitting...
5
05/22/2021 23:00:38
(200, 10528) (200, 1813) (200, 8650) (200, 65)
Fitting...
6
05/22/2021 23:00:41
(200, 6574) (200, 1813) (200, 4696) (200, 65)
Fitting...
7
05/22/2021 23:00:45
(200, 13290) (200, 1813) (200, 11412) (200, 65)
Fitting...
8
05/22/2021 23:00:50
(200, 8594) (200, 1813) (200, 6716) (200, 65)
Fitting...
9
05/22/2021 23:00:53
(200, 6024) (200, 1813) (200, 4146) (200, 65)
Fitting...
10
05/22/2021 23:00:57
(200, 11490) (200, 1813) (200, 9612) (200, 65)
Fitting...
11
05/22/2021 23:01:02
(200, 7344) (200, 1813) (200, 5466) (200, 65)
Fitting...
12
05/22/2021 23:01:05
(200, 13270) (200, 

In [ ]:
#LR
# {'lr__C': 100, 'cmin__df': 7, 'cngram__range': (3, 3), 'min__df': 5, 'ngram__range': 0}	0.65
df, vocab_set, vocab_dict = fit(data_tweets, 0, 5)
pickle.dump(df, open('/content/drive/My Drive/NLPproject/final/features/lr_df.pkl', 'wb'))
pickle.dump(vocab_set, open('/content/drive/My Drive/NLPproject/final/features/lr_vocab_set.pkl', 'wb'))
pickle.dump(vocab_dict, open('/content/drive/My Drive/NLPproject/final/features/lr_vocab_dict.pkl', 'wb'))

data_train_tr = transform(data_tweets, 0, df, vocab_set, vocab_dict)
char_vectorizer = TfidfVectorizer(ngram_range = (3, 3), min_df = 7, use_idf = True, smooth_idf = True, sublinear_tf = True, analyzer = 'char_wb')
data_train_char_tr = char_vectorizer.fit_transform(data_tweets_char)
pickle.dump(char_vectorizer, open('/content/drive/My Drive/NLPproject/final/features/lr_char_vectorizer.pkl', 'wb'))

f_data_train_tr = combine(data_train_tr, data_train_char_tr)
clr = LogisticRegression(C = 100, penalty = 'l2', solver = 'liblinear', fit_intercept = False, verbose = 0, random_state = 5)
clr.fit(f_data_train_tr, labels)
pickle.dump(clr, open('/content/drive/My Drive/NLPproject/final/models/lr_1.sav', 'wb'))

In [ ]:
# -----------------------------------------------------------------------------------------------------------------------------
# -----------------------------------------------------------------------------------------------------------------------------
# -----------------------------------------------------------------------------------------------------------------------------

data = pd.read_csv('/content/drive/My Drive/NLPproject/data/prepared_data_v7.tsv', delimiter = '\t', encoding = 'utf-8', converters = {'Tweets': eval})
data_tweets = data["Tweets"]
labels = data["Hate_speech"]

data_char = pd.read_csv('/content/drive/My Drive/NLPproject/data/concatenated_tweets_data.tsv', delimiter = '\t', encoding = 'utf-8')
data_tweets_char = data_char["Tweets"]

grid = ParameterGrid({"ngram__range":[1, 2, 0], "min__df": [4, 5, 6], "cngram__range":[(2, 2), (3, 3), (4, 4)], "cmin__df":[4, 5, 6], "rf__n_estimators":[100, 300, 400], "rf__min_samples_leaf":[7, 8, 9, 10]})
#svm_v1m_df = pd.DataFrame()
clr_df = pd.DataFrame()
params_list = []
scores_list = []
set_stopwords = set(stopwords.words('english'))
pd.set_option("display.max_rows", None, "display.max_columns", None)

counter = 1
print(time.strftime("%m/%d/%Y %H:%M:%S"))
for params in grid:
  clr = RandomForestClassifier(n_estimators = params['rf__n_estimators'], min_samples_leaf = params['rf__min_samples_leaf'], criterion = 'gini', random_state = 0, oob_score = True)

  df, vocab_set, vocab_dict = fit(data_tweets, params["ngram__range"], params["min__df"])
  data_train_tr = transform(data_tweets, params["ngram__range"], df, vocab_set, vocab_dict)
  char_vectorizer = TfidfVectorizer(ngram_range = params["cngram__range"], min_df = params["cmin__df"], use_idf = True, smooth_idf = True, sublinear_tf = True, analyzer = 'char_wb')
  data_train_char_tr = char_vectorizer.fit_transform(data_tweets_char)
  #f_data_train_tr = combine(data_train_tr, data_train_char_tr, train_index)
  f_data_train_tr = combine(data_train_tr, data_train_char_tr)
  print(f_data_train_tr.shape, data_train_char_tr.shape, data_train_tr.shape, td_matrix.shape)

  print("Fitting...")
  clr.fit(f_data_train_tr, labels)
  params_list.append(params)
  scores_list.append(clr.oob_score_)
  print(counter)
  print(time.strftime("%m/%d/%Y %H:%M:%S"))
  counter += 1

clr_df['params'] = params_list
clr_df['scores'] = scores_list
clr_df.to_csv('/content/drive/My Drive/NLPproject/results/F_rf1.tsv', sep = '\t', index = False)

05/22/2021 21:10:07
(200, 8789) (200, 2156) (200, 6568) (200, 65)
Fitting...
1
05/22/2021 21:10:10
(200, 8789) (200, 2156) (200, 6568) (200, 65)
Fitting...
2
05/22/2021 21:10:14
(200, 8789) (200, 2156) (200, 6568) (200, 65)
Fitting...
3
05/22/2021 21:10:18
(200, 8789) (200, 2156) (200, 6568) (200, 65)
Fitting...
4
05/22/2021 21:10:21
(200, 8789) (200, 2156) (200, 6568) (200, 65)
Fitting...
5
05/22/2021 21:10:25
(200, 8789) (200, 2156) (200, 6568) (200, 65)
Fitting...
6
05/22/2021 21:10:29
(200, 8789) (200, 2156) (200, 6568) (200, 65)
Fitting...
7
05/22/2021 21:10:32
(200, 8789) (200, 2156) (200, 6568) (200, 65)
Fitting...
8
05/22/2021 21:10:35
(200, 8789) (200, 2156) (200, 6568) (200, 65)
Fitting...
9
05/22/2021 21:10:39
(200, 8789) (200, 2156) (200, 6568) (200, 65)
Fitting...
10
05/22/2021 21:10:42
(200, 8789) (200, 2156) (200, 6568) (200, 65)
Fitting...
11
05/22/2021 21:10:46
(200, 8789) (200, 2156) (200, 6568) (200, 65)
Fitting...
12
05/22/2021 21:10:50
(200, 14005) (200, 2156) (200

In [ ]:
#RF
# {'cmin__df': 6, 'cngram__range': (4, 4), 'min__df': 4, 'ngram__range': 1, 'rf__min_samples_leaf': 9, 'rf__n_estimators': 400}	0.69
df, vocab_set, vocab_dict = fit(data_tweets, 1, 4)
pickle.dump(df, open('/content/drive/My Drive/NLPproject/final/features/rf_df.pkl', 'wb'))
pickle.dump(vocab_set, open('/content/drive/My Drive/NLPproject/final/features/rf_vocab_set.pkl', 'wb'))
pickle.dump(vocab_dict, open('/content/drive/My Drive/NLPproject/final/features/rf_vocab_dict.pkl', 'wb'))

data_train_tr = transform(data_tweets, 1, df, vocab_set, vocab_dict)
char_vectorizer = TfidfVectorizer(ngram_range = (4, 4), min_df = 6, use_idf = True, smooth_idf = True, sublinear_tf = True, analyzer = 'char_wb')
data_train_char_tr = char_vectorizer.fit_transform(data_tweets_char)
pickle.dump(char_vectorizer, open('/content/drive/My Drive/NLPproject/final/features/rf_char_vectorizer.pkl', 'wb'))

f_data_train_tr = combine(data_train_tr, data_train_char_tr)
clr = RandomForestClassifier(n_estimators = 400, min_samples_leaf = 9, criterion = 'gini', random_state = 0, oob_score = True)
clr.fit(f_data_train_tr, labels)
pickle.dump(clr, open('/content/drive/My Drive/NLPproject/final/models/rf_1.sav', 'wb'))


---

---

---

---

---
---


---
---

---

---

---

---
---


---

---



---
---



In [ ]:
def combine2(m1, m2, indexes):
  word_char = sparse.hstack((m1, m2)).tocsr()
  td_mat_slice = td_matrix[indexes, :]
  result = sparse.hstack((word_char, td_mat_slice)).tocsr()
  return result

In [ ]:
# -----------------------------------------------------------------------------------------------------------------------------
# -----------------------------------------------------------------------------------------------------------------------------
# -----------------------------------------------------------------------------------------------------------------------------

data = pd.read_csv('/content/drive/My Drive/NLPproject/data/prepared_data_v7.tsv', delimiter = '\t', encoding = 'utf-8', converters = {'Tweets': eval})
data_tweets = data["Tweets"]
labels = data["Hate_speech"]

data_char = pd.read_csv('/content/drive/My Drive/NLPproject/data/concatenated_tweets_data.tsv', delimiter = '\t', encoding = 'utf-8')
data_tweets_char = data_char["Tweets"]

clr_df = pd.DataFrame()
set_stopwords = set(stopwords.words('english'))


print(time.strftime("%m/%d/%Y %H:%M:%S"))
cv = StratifiedKFold(5, shuffle = True, random_state = 5)
results = []
for train_index, test_index in cv.split(data_tweets, labels):
  preds = pd.DataFrame()
  preds['ID'] = data["ID"][test_index].values
  labels_train, labels_test = labels[train_index], labels[test_index]
  data_train, data_test = data_tweets[train_index], data_tweets[test_index]
  data_train_char, data_test_char = data_tweets_char[train_index], data_tweets_char[test_index]

  df, vocab_set, vocab_dict = fit(data_train, 1, 5)
  data_train_tr = transform(data_train, 1, df, vocab_set, vocab_dict)
  char_vectorizer = TfidfVectorizer(ngram_range = (2, 2), min_df = 5, use_idf = True, smooth_idf = True, sublinear_tf = True, analyzer = 'char_wb')
  data_train_char_tr = char_vectorizer.fit_transform(data_train_char)
  f_data_train_tr = combine2(data_train_tr, data_train_char_tr, train_index)
  print(f_data_train_tr.shape, data_train_char_tr.shape, data_train_tr.shape, td_matrix.shape)

  svm_1 = svm.SVC(C = 1, kernel = 'linear', random_state = 5, probability = True, verbose = False)    
  svm_1.fit(f_data_train_tr, labels_train)

  data_test_tr = transform(data_test, 1, df, vocab_set, vocab_dict)
  data_test_char_tr = char_vectorizer.transform(data_test_char)
  f_data_test_tr = combine2(data_test_tr, data_test_char_tr, test_index)

  preds["preds"] = svm_1.predict_proba(f_data_test_tr)[:,1]
  results.append(preds)

#print(results)
results_pd = pd.concat(results)
#print(results_pd)
  
merged = pd.merge(results_pd, data, how = 'inner', on = 'ID')
merged.to_csv('/content/drive/My Drive/NLPproject/final/datasets/svm_dataset.tsv', sep = '\t', index = False)
final_data = merged.drop(["ID", "Hate_speech", "Tweets"], axis = 'columns')

pipeline = Pipeline([('svm', svm.SVC(kernel = 'linear', random_state = 5, verbose = False))])
parameters = {"svm__C":[1, 10, 100, 1000]}
grid_search = GridSearchCV(pipeline, parameters, cv = StratifiedKFold(5, shuffle = True, random_state = 0), n_jobs = -1)

print("Fitting...")
svm_2 = grid_search.fit(final_data.iloc[:,:], merged.iloc[:,2:3])

print(time.strftime("%m/%d/%Y %H:%M:%S"))


#print(test_vect)
clr_df['params'] = svm_2.cv_results_['params']
clr_df['scores'] = svm_2.cv_results_['mean_test_score']
clr_df.to_csv('/content/drive/My Drive/NLPproject/results/F_svm2.tsv', sep = '\t', index = False)

05/23/2021 01:49:00
(160, 6489) (160, 1800) (160, 4624) (200, 65)
(160, 6607) (160, 1786) (160, 4756) (200, 65)
(160, 6576) (160, 1793) (160, 4718) (200, 65)
(160, 6490) (160, 1808) (160, 4617) (200, 65)
(160, 6498) (160, 1790) (160, 4643) (200, 65)
Fitting...
05/23/2021 01:49:35


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
dataset = pd.read_csv('/content/drive/My Drive/NLPproject/final/datasets/svm_dataset.tsv', delimiter = '\t', encoding = 'utf-8')
final_data = dataset.drop(["ID", "Hate_speech", "Tweets"], axis = 'columns')
clr = svm.SVC(C = 1, kernel = 'linear', random_state = 5, verbose = False, probability = True)
clr.fit(final_data.iloc[:,:], dataset.iloc[:,2:3])
pickle.dump(clr, open('/content/drive/My Drive/NLPproject/final/models/svm_2.sav', 'wb'))

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:

# -----------------------------------------------------------------------------------------------------------------------------
# -----------------------------------------------------------------------------------------------------------------------------
# -----------------------------------------------------------------------------------------------------------------------------

data = pd.read_csv('/content/drive/My Drive/NLPproject/data/prepared_data_v7.tsv', delimiter = '\t', encoding = 'utf-8', converters = {'Tweets': eval})
data_tweets = data["Tweets"]
labels = data["Hate_speech"]

data_char = pd.read_csv('/content/drive/My Drive/NLPproject/data/concatenated_tweets_data.tsv', delimiter = '\t', encoding = 'utf-8')
data_tweets_char = data_char["Tweets"]

clr_df = pd.DataFrame()
set_stopwords = set(stopwords.words('english'))


print(time.strftime("%m/%d/%Y %H:%M:%S"))
cv = StratifiedKFold(5, shuffle = True, random_state = 5)
results = []
for train_index, test_index in cv.split(data_tweets, labels):
  preds = pd.DataFrame()
  preds['ID'] = data["ID"][test_index].values
  labels_train, labels_test = labels[train_index], labels[test_index]
  data_train, data_test = data_tweets[train_index], data_tweets[test_index]
  data_train_char, data_test_char = data_tweets_char[train_index], data_tweets_char[test_index]

  df, vocab_set, vocab_dict = fit(data_train, 0, 5)
  data_train_tr = transform(data_train, 0, df, vocab_set, vocab_dict)
  char_vectorizer = TfidfVectorizer(ngram_range = (3, 3), min_df = 7, use_idf = True, smooth_idf = True, sublinear_tf = True, analyzer = 'char_wb')
  data_train_char_tr = char_vectorizer.fit_transform(data_train_char)
  f_data_train_tr = combine2(data_train_tr, data_train_char_tr, train_index)
  print(f_data_train_tr.shape, data_train_char_tr.shape, data_train_tr.shape, td_matrix.shape)

  lr_1 = LogisticRegression(C = 100, penalty = 'l2', solver = 'liblinear', fit_intercept = False, verbose = 0, random_state = 5)  
  lr_1.fit(f_data_train_tr, labels_train)

  data_test_tr = transform(data_test, 0, df, vocab_set, vocab_dict)
  data_test_char_tr = char_vectorizer.transform(data_test_char)
  f_data_test_tr = combine2(data_test_tr, data_test_char_tr, test_index)

  preds["preds"] = lr_1.predict_proba(f_data_test_tr)[:,1]
  results.append(preds)

#print(results)
results_pd = pd.concat(results)
#print(results_pd)
  
merged = pd.merge(results_pd, data, how = 'inner', on = 'ID')
merged.to_csv('/content/drive/My Drive/NLPproject/final/datasets/lr_dataset.tsv', sep = '\t', index = False)
final_data = merged.drop(["ID", "Hate_speech", "Tweets"], axis = 'columns')

pipeline = Pipeline([('lr', LogisticRegression(penalty = 'l2', solver = 'liblinear', verbose = 0, random_state = 5))])
parameters = {"lr__C":[.1, 1, 10, 100, 1000]}
grid_search = GridSearchCV(pipeline, parameters, cv = StratifiedKFold(5, shuffle = True, random_state = 0), n_jobs = -1)

print("Fitting...")
lr_2 = grid_search.fit(final_data.iloc[:,:], merged.iloc[:,2:3])

print(time.strftime("%m/%d/%Y %H:%M:%S"))


#print(test_vect)
clr_df['params'] = lr_2.cv_results_['params']
clr_df['scores'] = lr_2.cv_results_['mean_test_score']
clr_df.to_csv('/content/drive/My Drive/NLPproject/results/F_lr2.tsv', sep = '\t', index = False)

05/23/2021 01:50:56
(160, 17031) (160, 5640) (160, 11326) (200, 65)
(160, 17355) (160, 5678) (160, 11612) (200, 65)
(160, 17200) (160, 5684) (160, 11451) (200, 65)
(160, 16950) (160, 5622) (160, 11263) (200, 65)
(160, 17034) (160, 5597) (160, 11372) (200, 65)
Fitting...
05/23/2021 01:51:17


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
dataset = pd.read_csv('/content/drive/My Drive/NLPproject/final/datasets/lr_dataset.tsv', delimiter = '\t', encoding = 'utf-8')
final_data = dataset.drop(["ID", "Hate_speech", "Tweets"], axis = 'columns')
clr = LogisticRegression(C = 10, penalty = 'l2', solver = 'liblinear', fit_intercept = False, verbose = 0, random_state = 5)  
clr.fit(final_data.iloc[:,:], dataset.iloc[:,2:3])
pickle.dump(clr, open('/content/drive/My Drive/NLPproject/final/models/lr_2.sav', 'wb'))

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
# -----------------------------------------------------------------------------------------------------------------------------
# -----------------------------------------------------------------------------------------------------------------------------
# -----------------------------------------------------------------------------------------------------------------------------

data = pd.read_csv('/content/drive/My Drive/NLPproject/data/prepared_data_v7.tsv', delimiter = '\t', encoding = 'utf-8', converters = {'Tweets': eval})
data_tweets = data["Tweets"]
labels = data["Hate_speech"]

data_char = pd.read_csv('/content/drive/My Drive/NLPproject/data/concatenated_tweets_data.tsv', delimiter = '\t', encoding = 'utf-8')
data_tweets_char = data_char["Tweets"]

clr_df = pd.DataFrame()
set_stopwords = set(stopwords.words('english'))


print(time.strftime("%m/%d/%Y %H:%M:%S"))
cv = StratifiedKFold(5, shuffle = True)
results = []
for train_index, test_index in cv.split(data_tweets, labels):
  preds = pd.DataFrame()
  preds['ID'] = data["ID"][test_index].values
  labels_train, labels_test = labels[train_index], labels[test_index]
  data_train, data_test = data_tweets[train_index], data_tweets[test_index]
  data_train_char, data_test_char = data_tweets_char[train_index], data_tweets_char[test_index]

  df, vocab_set, vocab_dict = fit(data_train, 1, 4)
  data_train_tr = transform(data_train, 1, df, vocab_set, vocab_dict)
  char_vectorizer = TfidfVectorizer(ngram_range = (4, 4), min_df = 6, use_idf = True, smooth_idf = True, sublinear_tf = True, analyzer = 'char_wb')
  data_train_char_tr = char_vectorizer.fit_transform(data_train_char)
  f_data_train_tr = combine2(data_train_tr, data_train_char_tr, train_index)
  print(f_data_train_tr.shape, data_train_char_tr.shape, data_train_tr.shape, td_matrix.shape)

  rf_1 = RandomForestClassifier(n_estimators = 400, min_samples_leaf = 9, criterion = 'gini', random_state = 0, oob_score = True)  
  rf_1.fit(f_data_train_tr, labels_train)

  data_test_tr = transform(data_test, 1, df, vocab_set, vocab_dict)
  data_test_char_tr = char_vectorizer.transform(data_test_char)
  f_data_test_tr = combine2(data_test_tr, data_test_char_tr, test_index)

  preds["preds"] = rf_1.predict_proba(f_data_test_tr)[:,1]
  results.append(preds)

#print(results)
results_pd = pd.concat(results)
#print(results_pd)
  
merged = pd.merge(results_pd, data, how = 'inner', on = 'ID')
merged.to_csv('/content/drive/My Drive/NLPproject/final/datasets/rf_dataset.tsv', sep = '\t', index = False)
final_data = merged.drop(["ID", "Hate_speech", "Tweets"], axis = 'columns')

grid = ParameterGrid({"rf__n_estimators":[100, 300, 400], "rf__min_samples_leaf":[5, 6, 7, 8, 9, 10]})

print("Fitting...")
counter = 1
scores_list = []
params_list = []
for params in grid:
  rf_2 = RandomForestClassifier(n_estimators = params['rf__n_estimators'], min_samples_leaf = params['rf__min_samples_leaf'], criterion = 'gini', random_state = 0, oob_score = True)
  rf_2.fit(final_data.iloc[:,:], merged.iloc[:,2:3])
  params_list.append(params)
  scores_list.append(rf_2.oob_score_)
  print(counter)
  print(time.strftime("%m/%d/%Y %H:%M:%S"))
  counter += 1
svm_2 = grid_search.fit(final_data.iloc[:,:], merged.iloc[:,2:3])

#print(test_vect)
clr_df['params'] = params_list
clr_df['scores'] = scores_list
clr_df.to_csv('/content/drive/My Drive/NLPproject/results/F_rf2.tsv', sep = '\t', index = False)

05/22/2021 23:33:29
(160, 18255) (160, 12609) (160, 5581) (200, 65)
(160, 18589) (160, 12800) (160, 5724) (200, 65)
(160, 18320) (160, 12658) (160, 5597) (200, 65)
(160, 18483) (160, 12761) (160, 5657) (200, 65)
(160, 18644) (160, 12890) (160, 5689) (200, 65)
Fitting...
1
05/22/2021 23:33:52


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:68: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:68: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


2
05/22/2021 23:33:53


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:68: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


3
05/22/2021 23:33:53
4
05/22/2021 23:33:54


...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:68: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:68: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


17
05/22/2021 23:34:00


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:68: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


18
05/22/2021 23:34:00


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
dataset = pd.read_csv('/content/drive/My Drive/NLPproject/final/datasets/rf_dataset.tsv', delimiter = '\t', encoding = 'utf-8')
final_data = dataset.drop(["ID", "Hate_speech", "Tweets"], axis = 'columns')
clr = RandomForestClassifier(n_estimators = 300, min_samples_leaf = 8, criterion = 'gini', random_state = 0, oob_score = True)
clr.fit(final_data.iloc[:,:], dataset.iloc[:,2:3])
pickle.dump(clr, open('/content/drive/My Drive/NLPproject/final/models/rf_2.sav', 'wb'))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """



---

---

---

---

---
---


---
---

---

---

---

---
---


---

---



---
---



In [ ]:
rf_dataset = pd.read_csv('/content/drive/My Drive/NLPproject/final/datasets/rf_dataset.tsv', delimiter = '\t', encoding = 'utf-8')
lr_dataset = pd.read_csv('/content/drive/My Drive/NLPproject/final/datasets/lr_dataset.tsv', delimiter = '\t', encoding = 'utf-8')
svm_dataset = pd.read_csv('/content/drive/My Drive/NLPproject/final/datasets/svm_dataset.tsv', delimiter = '\t', encoding = 'utf-8')

rf_dataset = rf_dataset.sort_values(by = ['ID']).reset_index(drop=True)
#print(rf_dataset.head())
#print("-----------------------------")
lr_dataset = lr_dataset.sort_values(by = ['ID']).reset_index(drop=True)
#print(lr_dataset.head())
#print("-----------------------------")
svm_dataset = svm_dataset.sort_values(by = ['ID']).reset_index(drop=True)
#print(svm_dataset.head())
#print("-----------------------------")

In [ ]:
tr_split = svm_dataset["ID"]
te_split = svm_dataset["Hate_speech"]

svm_2 = svm.SVC(C = 1, kernel = 'linear', random_state = 5, verbose = False, probability = True)
lr_2 = LogisticRegression(C = 10, penalty = 'l2', solver = 'liblinear', fit_intercept = False, verbose = 0, random_state = 5)
rf_2 = RandomForestClassifier(n_estimators = 300, min_samples_leaf = 8, criterion = 'gini', random_state = 0, oob_score = True)

cv = StratifiedKFold(5, shuffle = True)
results = []
for train_index, test_index in cv.split(tr_split, te_split):
  preds = pd.DataFrame()
  #preds['ID'] = svm_dataset["ID"][test_index].values
  preds['Truth'] = svm_dataset["Hate_speech"][test_index].values
  #print(preds)
  #print("-----------------------------")
  #print(svm_dataset["ID"][test_index].values)
  #print("-----------------------------")

  #SVM
  svm_train, svm_test = svm_dataset.iloc[train_index], svm_dataset.iloc[test_index]
  svm_train_data = svm_train.drop(["ID", "Hate_speech", "Tweets"], axis = 'columns')
  svm_2.fit(svm_train_data.iloc[:,:], svm_train.iloc[:,2:3])

  svm_test_data = svm_test.drop(["ID", "Hate_speech", "Tweets"], axis = 'columns')
  preds["svm"] = svm_2.predict_proba(svm_test_data)[:,1]

  #LR
  lr_train, lr_test = lr_dataset.iloc[train_index], lr_dataset.iloc[test_index]
  lr_train_data = lr_train.drop(["ID", "Hate_speech", "Tweets"], axis = 'columns')
  lr_2.fit(lr_train_data.iloc[:,:], lr_train.iloc[:,2:3])

  lr_test_data = lr_test.drop(["ID", "Hate_speech", "Tweets"], axis = 'columns')
  preds["lr"] = lr_2.predict_proba(lr_test_data)[:,1]
  
  #RF
  rf_train, rf_test = rf_dataset.iloc[train_index], rf_dataset.iloc[test_index]
  rf_train_data = rf_train.drop(["ID", "Hate_speech", "Tweets"], axis = 'columns')
  rf_2.fit(rf_train_data.iloc[:,:], rf_train.iloc[:,2:3])

  rf_test_data = rf_test.drop(["ID", "Hate_speech", "Tweets"], axis = 'columns')
  preds["rf"] = rf_2.predict_proba(rf_test_data)[:,1]
  
  #print(preds)
  print("-----------------------------")
  results.append(preds)

#print(results)
results_concat = pd.concat(results)
print(results_concat)
results_concat.to_csv('/content/drive/My Drive/NLPproject/final/datasets/ensemble_dataset_tr.tsv', sep = '\t', index = False)


cv = StratifiedKFold(5, shuffle = True)
results = []
for train_index, test_index in cv.split(tr_split, te_split):
  preds = pd.DataFrame()
  #preds['ID'] = svm_dataset["ID"][test_index].values
  preds['Truth'] = svm_dataset["Hate_speech"][test_index].values
  #print(preds)
  #print("-----------------------------")
  #print(svm_dataset["ID"][test_index].values)
  #print("-----------------------------")

  #SVM
  svm_train, svm_test = svm_dataset.iloc[train_index], svm_dataset.iloc[test_index]
  svm_train_data = svm_train.drop(["ID", "Hate_speech", "Tweets"], axis = 'columns')
  svm_2.fit(svm_train_data.iloc[:,:], svm_train.iloc[:,2:3])

  svm_test_data = svm_test.drop(["ID", "Hate_speech", "Tweets"], axis = 'columns')
  preds["svm"] = svm_2.predict_proba(svm_test_data)[:,1]

  #LR
  lr_train, lr_test = lr_dataset.iloc[train_index], lr_dataset.iloc[test_index]
  lr_train_data = lr_train.drop(["ID", "Hate_speech", "Tweets"], axis = 'columns')
  lr_2.fit(lr_train_data.iloc[:,:], lr_train.iloc[:,2:3])

  lr_test_data = lr_test.drop(["ID", "Hate_speech", "Tweets"], axis = 'columns')
  preds["lr"] = lr_2.predict_proba(lr_test_data)[:,1]
  
  #RF
  rf_train, rf_test = rf_dataset.iloc[train_index], rf_dataset.iloc[test_index]
  rf_train_data = rf_train.drop(["ID", "Hate_speech", "Tweets"], axis = 'columns')
  rf_2.fit(rf_train_data.iloc[:,:], rf_train.iloc[:,2:3])

  rf_test_data = rf_test.drop(["ID", "Hate_speech", "Tweets"], axis = 'columns')
  preds["rf"] = rf_2.predict_proba(rf_test_data)[:,1]
  
  #print(preds)
  print("-----------------------------")
  results.append(preds)

#print(results)
results_concat = pd.concat(results)
print(results_concat)
results_concat.to_csv('/content/drive/My Drive/NLPproject/final/datasets/ensemble_dataset_dev.tsv', sep = '\t', index = False)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:38: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


-----------------------------


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:38: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


-----------------------------


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:38: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


...

-----------------------------


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:83: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


-----------------------------


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:83: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


-----------------------------
    Truth       svm        lr        rf
0       1  0.835124  0.999478  0.629842
1       1  0.580193  0.820995  0.529718
2       0  0.489281  0.485763  0.589995
3       0  0.508160  0.603642  0.330788
4       0  0.467402  0.303586  0.432933
5       0  0.467328  0.321667  0.490033
6       1  0.824303  0.999263  0.715750
7       0  0.427265  0.234822  0.317424
8       1  0.445533  0.235118  0.474342
9       0  0.425800  0.254874  0.495557
10      0  0.526698  0.694062  0.714323
11      0  0.550381  0.690253  0.503517
12      0  0.542665  0.775191  0.773549
13      0  0.500000  0.420008  0.717630
14      0  0.527925  0.725873  0.587218
15      1  0.553887  0.552291  0.609646
16      0  0.478253  0.520129  0.744439
17      1  0.419610  0.245201  0.482076
18      0  0.547322  0.611153  0.539825
19      1  0.479020  0.434471  0.571780
20      1  0.684879  0.966654  0.606546
21      1  0.616773  0.921973  0.765196
22      1  0.461157  0.408426  0.633227
23      1 

In [ ]:
#load DF-s
ensemble_tr = pd.read_csv('/content/drive/My Drive/NLPproject/final/datasets/ensemble_dataset_tr.tsv', delimiter = '\t', encoding = 'utf-8')
ensemble_dev = pd.read_csv('/content/drive/My Drive/NLPproject/final/datasets/ensemble_dataset_dev.tsv', delimiter = '\t', encoding = 'utf-8')

# MEAN
print(classification_report(ensemble_tr["Truth"], ensemble_tr.iloc[:,1:].mean(axis = 1)>=0.5))
print(confusion_matrix(ensemble_tr["Truth"], ensemble_tr.iloc[:,1:].mean(axis = 1)>=0.5))

print(classification_report(ensemble_dev["Truth"], ensemble_dev.iloc[:,1:].mean(axis = 1)>=0.5))
print(confusion_matrix(ensemble_dev["Truth"], ensemble_dev.iloc[:,1:].mean(axis = 1)>=0.5))

# MAJORITY
print(classification_report(ensemble_tr["Truth"], (ensemble_tr.iloc[:,1:]>0.5).mean(1)>0.5))
print(confusion_matrix(ensemble_tr["Truth"], (ensemble_tr.iloc[:,1:]>0.5).mean(1)>0.5))

print(classification_report(ensemble_dev["Truth"], (ensemble_dev.iloc[:,1:]>0.5).mean(1)>0.5))
print(confusion_matrix(ensemble_dev["Truth"], (ensemble_dev.iloc[:,1:]>0.5).mean(1)>0.5))

# LOGREG
acc_scorer = make_scorer(accuracy_score)
scoring = {'Accuracy': acc_scorer}
params = {'solver' : ['saga'],
          'penalty' : ['elasticnet'],
          'C' : [0, 0.1, 0.2, 0.4, 0.7, 0.9, 1, 1.2, 1.5, 2],
          'l1_ratio' : [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]}

logreg_clf = LogisticRegression()
logreg = GridSearchCV(logreg_clf, param_grid = params, scoring = scoring, refit = "Accuracy", return_train_score = True, cv = 5, verbose = 1, n_jobs = -1)
logreg.fit(ensemble_tr.iloc[:,1:], ensemble_tr["Truth"])

#performance metrics and hyperpars
print(logreg.best_score_, logreg.best_params_, "\n", logreg.best_estimator_, "\n", logreg.best_estimator_.coef_, logreg.best_estimator_.intercept_)
print(classification_report(ensemble_dev["Truth"], logreg.predict(ensemble_dev.iloc[:,1:])))
print(confusion_matrix(ensemble_dev["Truth"], logreg.predict(ensemble_dev.iloc[:,1:])))

# LINREG
params = {'alpha' : [1, 2, 3, 5, 6, 7, 8, 9, 10, 12, 15]}
linreg_clf = RidgeClassifier()
linreg = GridSearchCV(linreg_clf, param_grid = params, scoring = scoring, refit = "Accuracy", return_train_score = True, cv = 5, verbose = 1, n_jobs = -1)
linreg.fit(ensemble_tr.iloc[:,1:], ensemble_tr["Truth"])

#performance metrics and hyperpars
print(linreg.best_score_, linreg.best_params_, "\n", linreg.best_estimator_, "\n", linreg.best_estimator_.coef_, linreg.best_estimator_.intercept_)
print(classification_report(ensemble_dev["Truth"], linreg.predict(ensemble_dev.iloc[:,1:])))
print(confusion_matrix(ensemble_dev["Truth"], linreg.predict(ensemble_dev.iloc[:,1:])))

              precision    recall  f1-score   support

           0       0.56      0.55      0.56       100
           1       0.56      0.57      0.56       100

    accuracy                           0.56       200
   macro avg       0.56      0.56      0.56       200
weighted avg       0.56      0.56      0.56       200

[[55 45]
 [43 57]]
              precision    recall  f1-score   support

           0       0.59      0.62      0.60       100
           1       0.60      0.57      0.58       100

    accuracy                           0.59       200
   macro avg       0.60      0.59      0.59       200
weighted avg       0.60      0.59      0.59       200

[[62 38]
 [43 57]]
              precision    recall  f1-score   support

           0       0.56      0.58      0.57       100
           1       0.56      0.54      0.55       100

    accuracy                           0.56       200
   macro avg       0.56      0.56      0.56       200
weighted avg       0.56      0.56   

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done 491 tasks      | elapsed:    3.1s
[Parallel(n_jobs=-1)]: Done 550 out of 550 | elapsed:    3.3s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


0.5850000000000001 {'C': 0.2, 'l1_ratio': 0.3, 'penalty': 'elasticnet', 'solver': 'saga'} 
 LogisticRegression(C=0.2, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=0.3, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='elasticnet',
                   random_state=None, solver='saga', tol=0.0001, verbose=0,
                   warm_start=False) 
 [[0.0780938  0.43686383 0.22370768]] [-0.38634494]
              precision    recall  f1-score   support

           0       0.59      0.64      0.61       100
           1       0.60      0.55      0.58       100

    accuracy                           0.59       200
   macro avg       0.60      0.59      0.59       200
weighted avg       0.60      0.59      0.59       200

[[64 36]
 [45 55]]
Fitting 5 folds for each of 11 candidates, totalling 55 fits
0.5800000000000001 {'alpha': 1} 
 RidgeClassifier(alpha=1, class_weight=None, copy_X=True, fit_intercept=True,
     

[Parallel(n_jobs=-1)]: Done  55 out of  55 | elapsed:    0.3s finished


In [ ]:
# save out linear regression for aggregation
joblib.dump(linreg, '/content/drive/My Drive/NLPproject/final/models/ensemble_linreg')

['/content/drive/My Drive/NLPproject/final/models/ensemble_linreg']


---

---

---

---

---
---


---
---

---

---

---

---
---


---

---



---
---



In [ ]:
model_loc = "/content/drive/My Drive/NLPproject/final/models/"
vectorizers = "/content/drive/My Drive/NLPproject/final/features/"

b = "/content/drive/My Drive/NLPproject/final/testset_predictions/en/"
a = "/content/drive/My Drive/NLPproject/final/pan21-author-profiling-test-without-gold-for-participants.zip"

dir = '/'.join(a.split("/")[:-1])
dir_name = a.split("/")[-1].split('.')[0]

In [ ]:
with zipfile.ZipFile(a, 'r') as zip_ref:
    zip_ref.extractall(dir)

In [ ]:
# -----------------------------------------------------------------------------------------------------------------------------
# -----------------------------------------------------------------------------------------------------------------------------
# -----------------------------------------------------------------------------------------------------------------------------

pathlist = Path(dir + '/' + dir_name + '/en/').glob('*.xml')
id = []
tweets_raw_lists, tweets_raw_concat = [], []
for path in pathlist:
  head, tail = os.path.split(path)
  author = tail.split(".")[0]
  id.append(author)
  path_in_str = str(path)
  tree = ET.parse(path_in_str)
  docs = tree.getroot().getchildren()[0]
  tweets = []
  for tweet in docs:
    tweets.append(tweet.text)
  tweets_raw_lists.append(tweets)
  tweets_raw_concat.append(' '.join(tweets).lower())

lists_tweets_data = pd.DataFrame()
lists_tweets_data["ID"] = id
lists_tweets_data["Tweets"] = tweets_raw_lists 

concat_tweets_data = pd.DataFrame()
concat_tweets_data["ID"] = id
concat_tweets_data["Tweets"] = tweets_raw_concat

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: This method will be removed in future versions.  Use 'list(elem)' or iteration over elem instead.
  


In [ ]:
print(lists_tweets_data.head())
print(concat_tweets_data.head())
pd.set_option("display.max_rows", None, "display.max_columns", None)

                                 ID  \
0  033b741821a47b223952d00ad3a96e1e   
1  03ef536dad2f429dde26114bf382ac53   
2  059628dc5dc8b19d2a263766b6ca7622   
3  05e1527e1d7210f6da3a90d4c7e53dd7   
4  061a343a9faec7d73cedd1aedc59b07c   

                                              Tweets  
0  [Hit a open 3 or imma hit yo ass 😂 #URL#, Eric...  
1  [RT #USER#: god damn the ‘blue maga’ shit is s...  
2  [RT #USER#: No but, no, Very bitch made #URL#,...  
3  [RT #USER#: i rlly hate random sad days. havin...  
4  [Lets stop with no longer respecting Ibram Ken...  
                                 ID  \
0  033b741821a47b223952d00ad3a96e1e   
1  03ef536dad2f429dde26114bf382ac53   
2  059628dc5dc8b19d2a263766b6ca7622   
3  05e1527e1d7210f6da3a90d4c7e53dd7   
4  061a343a9faec7d73cedd1aedc59b07c   

                                              Tweets  
0  hit a open 3 or imma hit yo ass 😂 #url# eric g...  
1  rt #user#: god damn the ‘blue maga’ shit is so...  
2  rt #user#: no but, no very bitch

In [ ]:
# -----------------------------------------------------------------------------------------------------------------------------
# -----------------------------------------------------------------------------------------------------------------------------
# -----------------------------------------------------------------------------------------------------------------------------

def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return None

def negation(tokens):
  negations_booleans = []
  boolean = False
  for token in tokens:
    token_boolean = False
    if token in negations:
      if boolean:
        boolean = False
      else:
        boolean = True
    elif re.match(r'[\.:;!?]+$', token):
      boolean = False
    elif boolean:
      token_boolean = True
    negations_booleans.append(token_boolean)
  return negations_booleans

def rescale(value, min, max):
  scale = 1/(max - min)
  return (value - min) * scale

def negate_(value, negate):
  result = value
  if negate:
    if value > 0.5:
      result = value - ((value - 0.5) * 2)
    else:
      result = ((0.5 - value) * 2) + value
  return np.round(result, 3)

def sentiment_value(word, negate, wntag):
  synsets = wordnet.synsets(word, pos = wntag)
  if len(synsets) != 0:
    synset = synsets[0]
    swn_synset = sentiwordnet.senti_synset(synset.name())
    pos, neg = swn_synset.pos_score(), swn_synset.neg_score()
    score = pos - neg
    if score == 0:
      return True, -1
    else:
      result = np.round(rescale(score, -1., 1.), 3)
      return True, negate_(result, negate)

  sentiment = sentiment_dict.get(word)
  if sentiment is not None:
    if sentiment == 0.5:
      return True, -1
    return True, negate_(sentiment, negate)

  return False, "unknown"

def lemma_(word, wntag):
  if wntag is None:
    lemma = lem.lemmatize(word) 
  else:
    lemma = lem.lemmatize(word, pos = wntag)
  return lemma

def dict_emoticons():
  return {":\)+":"happy", ":\(+":"sad", ":[Dd]+":"happy", "=\)+":"happy", ":[pP]+":"playful", ":3+":"happy", 
          "[Xx][dD]+":"happy_laugh", ">:\(+":"angry", ":'\(+":"sad_cry", "[Xx][Pp]+":"playful"}

def emoticon(word):
  dict_emoticon_patterns = dict_emoticons()
  dict_list = list(dict_emoticon_patterns.keys())
  for emoticon in dict_list:
    if re.search(f"^{emoticon}", word):
      return re.sub(f"^{emoticon}", dict_emoticon_patterns[emoticon], word)
  return word

def demojify(word):
  new_string = []
  boolean = True
  for char in word:
    if char in UNICODE_EMOJI["en"]:
      new_string.append(' ' + char + ' ')
      boolean = False
    else:
      new_string.append(char)
  if boolean:
    return word
  new_string = re.sub(r' +', ' ', ''.join(new_string)).strip()
  return emoji.demojize(new_string)

def parse(sentence, nlp, td_matrix, sample):
  sentence = re.sub(r'[^0-9A-Za-z _!.?,();"\'\\%$&~/-]', '', sentence)
  sentence = re.sub(r' +', ' ', sentence).strip()
  doc = nlp(sentence)
  for sent in doc.sentences:
    typed_dependancies = [tup[1] for tup in sent.dependencies]
    for td in sent.dependencies:
      if td[1] == "root":
        continue
      if td[1] not in set_td:
        continue
      index = dict_td[td[1]]
      value = td_matrix[sample, index]
      td_matrix[sample, index] = value + 1
  return td_matrix

def prep_and_extraction(data, td_bool):
  
  all_tweets = data["Tweets"].tolist()
  cleaned_tweets = []

  sd_len_char_acc, sd_len_word_acc, min_len_char_acc, min_len_word_acc, max_len_char_acc, max_len_word_acc = [], [], [], [], [], []
  rng_len_char_acc, rng_len_word_acc, mean_len_char_acc, mean_len_word_acc = [], [], [], []
  rt_acc, url_acc, hash_acc, user_acc = [], [], [], []
  ttr_acc = []
  pos_acc, neg_acc, neu_acc, unk_acc, sent_mean_acc, sent_std_acc, hate_acc = [], [], [], [], [], [], []
  counter = 0
  if td_bool:
    nlp = stanza.Pipeline('en')
    typedep_matrix = np.zeros((data.shape[0], len(dict_td)), dtype = np.double)

  for account_feed in all_tweets:
    print(counter)
    cleaned_feed = []

    char_lenghts, word_lenghts, ttr_ = [], [], []
    rts, urls, hashs, users = 0, 0, 0, 0
    positives, negatives, neutrals, unknowns, hatefulls = 0, 0, 0, 0, 0
    overall_sent = []

    for tweet in account_feed:

      tweet = unescape(tweet)

      ttr_.extend(ld.flemmatize(tweet))
      char_lenghts.append(len(tweet))

      words = []
      tweet_list = tweet.split()

      word_lenghts.append(len(tweet_list))

      for word in tweet_list:
        new_word = contractions.fix(word)
        if new_word != "":
          words.append(new_word)
      tweet = ' '.join(words)
      
      words = []
      for_parsing = []
      tweet_list = tweet.split()

      for word in tweet_list:

        parsing_word = word
        new_word = demojify(word)

        if not new_word.isalnum():
          new_word = emoticon(new_word)
          
          if re.search("#USER#", new_word):
            new_word = re.sub("#USER#", "", new_word)
            parsing_word = re.sub("#USER#", "", parsing_word)
            users += 1
          
          if re.search("#URL#", new_word):
            new_word = re.sub("#URL#", "", new_word)
            parsing_word = re.sub("#URL#", "", parsing_word)
            urls += 1

          if re.search("#HASHTAG#", new_word):
            new_word = re.sub("#HASHTAG#", "", new_word)
            parsing_word = re.sub("#HASHTAG#", "", parsing_word)
            hashs += 1
        
        if re.match("RT$", new_word):
          new_word = re.sub("RT", "", new_word)
          parsing_word = re.sub("RT", "", parsing_word)
          rts += 1

        if new_word != "":
          words.append(new_word)
        for_parsing.append(parsing_word)
      tweet = ' '.join(words)

      if td_bool:
        typedep_matrix = parse(' '.join(for_parsing), nlp, typedep_matrix, counter)

      tweet = tweet.lower()
      
      sentiment = 0.
      sentences = sent_tokenize(tweet)
      for sent in sentences:

        sent = re.sub("_|-", " ", sent)
        words = word_tokenize(sent)
        tagged = nltk.pos_tag(words)

        stemmed_words = [ps.stem(word) for word in words]
        stemmed_tagged = nltk.pos_tag(stemmed_words)

        booleans = negation(words)

        for i in range(len(words)):
          
          word = words[i]
          word = re.sub('[^0-9a-z]', "", word)
          
          if word != '' and word not in set_stopwords:
            wntag = get_wordnet_pos(tagged[i][1])
            
            exists, score = sentiment_value(word, booleans[i], wntag)
            hatefull = word in hatefull_set

            if not exists:
              lemma = lemma_(word, wntag)
              exists, score = sentiment_value(lemma, booleans[i], wntag)
            if not hatefull:
              lemma = lemma_(word, wntag)
              hatefull = lemma in hatefull_set
            
            if not exists:
              exists, score = sentiment_value(stemmed_words[i], booleans[i], get_wordnet_pos(stemmed_tagged[i][1]))
            if not hatefull:
              hatefull = stemmed_words[i] in hatefull_set
              
            if not exists:
              if not hatefull:
                unknowns += 1
            else:
              if score == -1:
                neutrals += 1
              elif score > 0.5:
                positives += 1
                sentiment += score
              else:
                negatives += 1
                sentiment += score
              
            if hatefull:
              hatefulls += 1
      overall_sent.append(sentiment)

      tweet = re.sub('[^a-z _-]', "", tweet)

      words = []
      tweet_list = tweet.split()
      for word in tweet_list:

        new_word = re.sub('[0-9]+', 'num', word)

        if new_word != "":
          words.append(new_word)
      tweet = ' '.join(words)

      tweet = re.sub(r' +', ' ', tweet)

      cleaned_feed.append(tweet)
    cleaned_tweets.append(cleaned_feed)

    sd_len_char_acc.append(pstdev(char_lenghts))
    sd_len_word_acc.append(pstdev(word_lenghts))
    min_char = min(char_lenghts)
    min_len_char_acc.append(min_char)
    min_word = min(word_lenghts)
    min_len_word_acc.append(min_word)
    max_char = max(char_lenghts)
    max_len_char_acc.append(max_char)
    max_word = max(word_lenghts)
    max_len_word_acc.append(max_word)
    rng_len_char_acc.append(max_char - min_char)
    rng_len_word_acc.append(max_word - min_word)
    mean_len_char_acc.append(np.mean(char_lenghts))
    mean_len_word_acc.append(np.mean(word_lenghts))

    rt_acc.append(rts)
    url_acc.append(urls)
    hash_acc.append(hashs)
    user_acc.append(users)

    ttr_acc.append(ld.ttr(ttr_))

    pos_acc.append(positives)
    neg_acc.append(negatives)
    neu_acc.append(neutrals)
    unk_acc.append(unknowns)
    sent_std_acc.append(pstdev(overall_sent))
    sent_mean_acc.append(np.mean(overall_sent))
    hate_acc.append(hatefulls)

    counter += 1

  data["Tweets"] = cleaned_tweets

  stacked = np.column_stack((sd_len_char_acc, sd_len_word_acc, min_len_char_acc, min_len_word_acc, max_len_char_acc, max_len_word_acc, rng_len_char_acc, rng_len_word_acc, mean_len_char_acc, mean_len_word_acc, rt_acc, url_acc, hash_acc, user_acc, ttr_acc, pos_acc, neg_acc, neu_acc, unk_acc, sent_std_acc, sent_mean_acc, hate_acc))
  scaled = stan_scaler.transform(stacked)

  data["sd_len_char_acc"] = scaled[:, 0].tolist()
  data["sd_len_word_acc"] = scaled[:, 1].tolist()
  data["min_len_char_acc"] = scaled[:, 2].tolist()
  data["min_len_word_acc"] = scaled[:, 3].tolist()
  data["max_len_char_acc"] = scaled[:, 4].tolist()
  data["max_len_word_acc"] = scaled[:, 5].tolist()
  data["rng_len_char_acc"] = scaled[:, 6].tolist()
  data["rng_len_word_acc"] = scaled[:, 7].tolist()
  data["mean_len_char_acc"] = scaled[:, 8].tolist()
  data["mean_len_word_acc"] = scaled[:, 9].tolist()

  data["rt_acc"] = scaled[:, 10].tolist()
  data["url_acc"] = scaled[:, 11].tolist()
  data["hash_acc"] = scaled[:, 12].tolist()
  data["user_acc"] = scaled[:, 13].tolist()

  data["ttr_acc"] = scaled[:, 14].tolist()

  data["pos_acc"] = scaled[:, 15].tolist()
  data["neg_acc"] = scaled[:, 16].tolist()
  data["neu_acc"] = scaled[:, 17].tolist()
  data["unk_acc"] = scaled[:, 18].tolist()
  data["sent_std_acc"] = scaled[:, 19].tolist()
  data["sent_mean_acc"] = scaled[:, 20].tolist()
  data["hate_acc"] = scaled[:, 21].tolist()

  if td_bool:
    typedep_matrix = normalize(typedep_matrix, norm = 'l2', axis = 1)
    typedep_matrix = sparse.csr_matrix(typedep_matrix)
    return data, typedep_matrix
  else:
    return data



lem = WordNetLemmatizer()
ps = PorterStemmer()
set_stopwords = set(stopwords.words('english'))
negations = set(['not', 'never', 'none', 'nobody', 'nowhere', 'nothing', 'neither', 'no', 'noone'])

sentiment_file = open("/content/drive/My Drive/NLPproject/additional_stuff/sentiment_dict.pkl", "rb")
sentiment_dict = pickle.load(sentiment_file)

hatefull_df = pd.read_csv('/content/drive/My Drive/NLPproject/additional_stuff/hatebase_vocab.csv')
hatefull_set = set()
for term in hatefull_df['term']:
  hatefull_set.add(term.lower())

stan_scaler = pickle.load(open('/content/drive/My Drive/NLPproject/models/stan_scaler.pickle', "rb"))

td_matrix = 0
data = ''
str_path = vectorizers + 'td_matrix.pkl'
if os.path.exists(str_path):
  with open(str_path, 'rb') as f:
    td_matrix = pickle.load(f)
    data = prep_and_extraction(lists_tweets_data, False)
else:
  list_td = ['nmod:npmod', 'det:predet', 'obl:npmod', 'acl', 'acl:relcl', 'advcl', 'advmod', 'advmod:emph', 'advmod:lmod', 'amod', 'appos', 'aux', 'aux:pass', 'case', 'cc', 'cc:preconj', 'ccomp', 'clf', 'compound', 'compound:lvc', 'compound:prt', 'compound:redup', 'compound:svc', 'conj', 'cop', 'csubj', 'csubj:pass', 'dep', 'det', 'det:numgov', 'det:nummod', 'det:poss', 'discourse', 'dislocated', 'expl', 'expl:impers', 'expl:pass', 'expl:pv', 'fixed', 'flat', 'flat:foreign', 'flat:name', 'goeswith', 'iobj', 'list', 'mark', 'nmod', 'nmod:poss', 'nmod:tmod', 'nsubj', 'nsubj:pass', 'nummod', 'nummod:gov', 'obj', 'obl', 'obl:agent', 'obl:arg', 'obl:lmod', 'obl:tmod', 'orphan', 'parataxis', 'punct', 'reparandum', 'vocative', 'xcomp']
  dict_td = {}
  for i in range(len(list_td)):
    dict_td[list_td[i]] = i
  set_td = set(list_td)
  data, td_matrix = prep_and_extraction(lists_tweets_data, True)
  with open(str_path, 'wb') as f:
    pickle.dump(td_matrix, f)

print(td_matrix)
print(data.head())

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
  (0, 0)	0.002161951188760787
  (0, 1)	0.00648585356628236
  (0, 2)	0.008647804755043148
  (0, 3)	0.028105365453890228
  (0, 4)	0.04756292615273731
  (0, 5)	0.11674536419308248
  (0, 6)	0.3415882878242043
  (0, 9)	0.17079414391210215
  (0, 10)	0.00648585356628236
  (0, 11)	0.15566048559077664
  (0, 12)	0.03459121902017259
  (0, 13)	0.3005112152377494
  (0, 14)	0.06702048685158439
  (0, 16)	0.07134438922910596
  (0, 18)	0.16430829034581979
  (0, 20)	0.04756292615273731
  (0, 23)	0.07350634041786674
  (0, 24)	0.1470126808357335
  (0, 25)	0.004323902377521574
  (0, 27)	0.004323902377521574
  (0, 28)	0.34591219020172587
  (0, 32)	0.07350634041786674
  (0, 34)	0.010809755943803933
  (0, 38)	0.0021619511887

data e different, sega e bez hate_speach. prethodno bese so toa, taka da indeksite se izmesteni.


In [ ]:
# -----------------------------------------------------------------------------------------------------------------------------
# -----------------------------------------------------------------------------------------------------------------------------
# -----------------------------------------------------------------------------------------------------------------------------

def unigrams_tf(words, tf):
  for word in words:
    if word not in set_stopwords:
      tf[word] = tf.get(word, 0) + 1

def bigrams_tf(words, tf):
  previous = ''
  for word in words:
    if previous != '':
      bigram = previous + ' ' + word
      tf[bigram] = tf.get(bigram, 0) + 1
    previous = word

def both_tf(words, tf):
  previous = ''
  for word in words:
    if previous != '':
      bigram = previous + ' ' + word
      tf[bigram] = tf.get(bigram, 0) + 1
    if word not in set_stopwords:
      tf[word] = tf.get(word, 0) + 1
    previous = word

def build_tfidf(data, df, vocab_set, vocab_dict, n_gram):
  tfidf_matrix = np.zeros((data.shape[0], len(df)), dtype = np.double)

  sample = 0
  for account_feed in data:
    account_tf = {}

    for tweet in account_feed:
      words = re.split(r"[\s]+", tweet.strip())
      if n_gram == 0:
        both_tf(words, account_tf)
      elif n_gram == 1:
        unigrams_tf(words, account_tf)
      else:
        bigrams_tf(words, account_tf)
    
    for key, value in account_tf.items():
      #print(tf.items())
      if key not in vocab_set:
        continue
      index = vocab_dict[key]
      tf_idf = (1. + np.log(value)) * (np.log((1. + data.shape[0]) / (1. + df[key])) + 1.)
      tfidf_matrix[sample, index] = tf_idf
    sample += 1

  tfidf_matrix = normalize(tfidf_matrix, norm = 'l2', axis = 1)
  return tfidf_matrix

def transform(data, n_gram, df, vocab_set, vocab_dict):
  tfidf_matrix = build_tfidf(data, df, vocab_set, vocab_dict, n_gram)
  tfidf_matrix = sparse.csr_matrix(tfidf_matrix)
  #print(tfidf_matrix.getnnz() / np.prod(tfidf_matrix.shape), " %")
  return tfidf_matrix

#def combine(m1, m2, indexes):
def combine(m1, m2):
  word_char = sparse.hstack((m1, m2)).tocsr()
  #td_mat_slice = td_matrix[indexes, :]
  #result = sparse.hstack((word_char, td_mat_slice)).tocsr()
  result = sparse.hstack((word_char, td_matrix)).tocsr()
  return result

In [ ]:
data_tweets = data["Tweets"] ## kadesto imame id | tweets | rest...
data_tweets_char = concat_tweets_data["Tweets"] ## kadesto imame id | tweets

#SVM1
svm_df = pickle.load(open('/content/drive/My Drive/NLPproject/final/features/svm_df.pkl', "rb"))
svm_vocab_set = pickle.load(open('/content/drive/My Drive/NLPproject/final/features/svm_vocab_set.pkl', "rb"))
svm_vocab_dict = pickle.load(open('/content/drive/My Drive/NLPproject/final/features/svm_vocab_dict.pkl', "rb"))
data_test_tr = transform(data_tweets, 1, svm_df, svm_vocab_set, svm_vocab_dict)
svm_char_vectorizer = pickle.load(open('/content/drive/My Drive/NLPproject/final/features/svm_char_vectorizer.pkl', "rb"))
data_test_char_tr = svm_char_vectorizer.transform(data_tweets_char)
f_data_test_tr = combine(data_test_tr, data_test_char_tr)
print(f_data_test_tr)
print("-----------------------------")

svm_1 = pickle.load(open('/content/drive/My Drive/NLPproject/final/models/svm_1.sav', "rb"))
preds_svm_1 = svm_1.predict_proba(f_data_test_tr)[:, 1]
print(preds_svm_1)
print("-----------------------------")

#LR1
lr_df = pickle.load(open('/content/drive/My Drive/NLPproject/final/features/lr_df.pkl', "rb"))
lr_vocab_set = pickle.load(open('/content/drive/My Drive/NLPproject/final/features/lr_vocab_set.pkl', "rb"))
lr_vocab_dict = pickle.load(open('/content/drive/My Drive/NLPproject/final/features/lr_vocab_dict.pkl', "rb"))
data_test_tr = transform(data_tweets, 0, lr_df, lr_vocab_set, lr_vocab_dict)
lr_char_vectorizer = pickle.load(open('/content/drive/My Drive/NLPproject/final/features/lr_char_vectorizer.pkl', "rb"))
data_test_char_tr = lr_char_vectorizer.transform(data_tweets_char)
f_data_test_tr = combine(data_test_tr, data_test_char_tr)
print(f_data_test_tr)
print("-----------------------------")

lr_1 = pickle.load(open('/content/drive/My Drive/NLPproject/final/models/lr_1.sav', "rb"))
preds_lr_1 = lr_1.predict_proba(f_data_test_tr)[:, 1]
print(preds_lr_1)
print("-----------------------------")

#RF1
rf_df = pickle.load(open('/content/drive/My Drive/NLPproject/final/features/rf_df.pkl', "rb"))
rf_vocab_set = pickle.load(open('/content/drive/My Drive/NLPproject/final/features/rf_vocab_set.pkl', "rb"))
rf_vocab_dict = pickle.load(open('/content/drive/My Drive/NLPproject/final/features/rf_vocab_dict.pkl', "rb"))
data_test_tr = transform(data_tweets, 1, rf_df, rf_vocab_set, rf_vocab_dict)
rf_char_vectorizer = pickle.load(open('/content/drive/My Drive/NLPproject/final/features/rf_char_vectorizer.pkl', "rb"))
data_test_char_tr = rf_char_vectorizer.transform(data_tweets_char)
f_data_test_tr = combine(data_test_tr, data_test_char_tr)
print(f_data_test_tr)
print("-----------------------------")

rf_1 = pickle.load(open('/content/drive/My Drive/NLPproject/final/models/rf_1.sav', "rb"))
preds_rf_1 = rf_1.predict_proba(f_data_test_tr)[:, 1]
print(preds_rf_1)
print("-----------------------------")

#da spoime preds od prethodniov kod kako prv feature so rest od data(rest doagaat posle preds)
#preds + data[rest]

#SVM2
preds_svm_1 = pd.DataFrame({'preds': preds_svm_1})
data_svm_2 = pd.concat([preds_svm_1, data.iloc[:,2:]], axis = 1)
print(data_svm_2.head())
print("-----------------------------")
svm_2 = pickle.load(open('/content/drive/My Drive/NLPproject/final/models/svm_2.sav', "rb"))
preds_svm_2 = svm_2.predict_proba(data_svm_2)[:, 1]
print(preds_svm_2)
print("-----------------------------")

#LR2
preds_lr_1 = pd.DataFrame({'preds': preds_lr_1})
data_lr_2 = pd.concat([preds_lr_1, data.iloc[:,2:]], axis = 1)
print(data_lr_2.head())
print("-----------------------------")
lr_2 = pickle.load(open('/content/drive/My Drive/NLPproject/final/models/lr_2.sav', "rb"))
preds_lr_2 = lr_2.predict_proba(data_lr_2)[:, 1]
print(preds_lr_2)
print("-----------------------------")

#RF2
preds_rf_1 = pd.DataFrame({'preds': preds_rf_1})
data_rf_2 = pd.concat([preds_rf_1, data.iloc[:,2:]], axis = 1)
print(data_rf_2.head())
print("-----------------------------")
rf_2 = pickle.load(open('/content/drive/My Drive/NLPproject/final/models/rf_2.sav', "rb"))
preds_rf_2 = rf_2.predict_proba(data_rf_2)[:, 1]
print(preds_rf_2)
print("-----------------------------")

#META-LEARNER
meta_learner_data = pd.DataFrame()
meta_learner_data["svm"] = preds_svm_2
meta_learner_data["lr"] = preds_lr_2
meta_learner_data["rf"] = preds_rf_2
print(meta_learner_data.head())
print("-----------------------------")
#load model
meta_learner = joblib.load('/content/drive/My Drive/NLPproject/final/models/ensemble_linreg')
#predictions
final_preds = meta_learner.predict(meta_learner_data)
print(final_preds)
print("-----------------------------")

  (0, 0)	0.029992372805891992
  (0, 1)	0.013682893693392189
  (0, 8)	0.019146368292659262
  (0, 10)	0.013682893693392189
  (0, 13)	0.020638918983194633
  (0, 15)	0.02443862974050658
  (0, 16)	0.014961263907167661
  (0, 17)	0.015447169383179537
  (0, 22)	0.026061262717023945
  (0, 23)	0.015373530054770853
  (0, 35)	0.029927594608588997
  (0, 37)	0.039973171574269835
  (0, 39)	0.011453256631871935
  (0, 40)	0.014573145250808885
  (0, 41)	0.014379841243193055
  (0, 42)	0.010946061637350296
  (0, 52)	0.014237882089602586
  (0, 60)	0.020638918983194633
  (0, 71)	0.03494472748620113
  (0, 75)	0.0200073599347964
  (0, 85)	0.02468981432742173
  (0, 87)	0.0185517281969303
  (0, 91)	0.027420700169133293
  (0, 93)	0.023464284892342646
  (0, 97)	0.010060746799488018
  :	:
  (99, 7413)	0.15548388345750377
  (99, 7415)	0.024352897409006615
  (99, 7418)	0.11989118724434027
  (99, 7419)	0.15173728385611815
  (99, 7420)	0.0037465996013856333
  (99, 7423)	0.31096776691500755
  (99, 7427)	0.1779634810658

In [ ]:
for i in range(len(id)):
  a_id = str(id[i])
  pred = final_preds[i]   
  root = ET.Element("author", id = a_id, lang = "en", type = str(pred))
  tree = ET.ElementTree(root)
  tree.write(b + a_id + ".xml")